In [1]:
import os
import shutil
import argparse
import datetime
import pickle
import time

import numpy as np
import matplotlib as mpl
mpl.use('Agg')

import sys
sys.path.append('../')

import mxnet as mx
mx.random.seed(int(time.time()))

from utils import gpu_helper, gpu_exists, calc_topic_uniqueness, get_topic_words_decoder_weights, request_pmi, print_topic_with_scores, print_topics

from tqdm import tqdm

In [5]:
parser = argparse.ArgumentParser(description='Training WAE in MXNet')
parser.add_argument('-dom','--domain', type=str, default='wikitext-103', help='domain to run', required=False)
parser.add_argument('-data','--data_path', type=str, default='', help='file path for dataset', required=False)
parser.add_argument('-max_labels','--max_labels', type=int, default=100, help='max number of topics to specify as labels for a single training document', required=False)
parser.add_argument('-max_labeled_samples','--max_labeled_samples', type=int, default=10, help='max number of labeled samples per topic', required=False)
parser.add_argument('-label_seed','--label_seed', type=lambda x: int(x) if x != 'None' else None, default=None, help='random seed for subsampling the labeled dataset', required=False)
parser.add_argument('-mod','--model', type=str, default='dirichlet', help='model to use', required=False)
parser.add_argument('-desc','--description', type=str, default='', help='description for the experiment', required=False)
parser.add_argument('-alg','--algorithm', type=str, default='standard', help='algorithm to use for training: standard', required=False)
parser.add_argument('-bs','--batch_size', type=int, default=256, help='batch_size for training', required=False)
parser.add_argument('-opt','--optim', type=str, default='Adam', help='encoder training algorithm', required=False)
parser.add_argument('-lr','--learning_rate', type=float, default=1e-4, help='learning rate', required=False)
parser.add_argument('-l2','--weight_decay', type=float, default=0., help='weight decay', required=False)
parser.add_argument('-e_nh','--enc_n_hidden', type=int, nargs='+', default=[128], help='# of hidden units for encoder or list of hiddens for each layer', required=False)
parser.add_argument('-e_nl','--enc_n_layer', type=int, default=1, help='# of hidden layers for encoder, set to -1 if passing list of n_hiddens', required=False)
parser.add_argument('-e_nonlin','--enc_nonlinearity', type=str, default='sigmoid', help='type of nonlinearity for encoder', required=False)
parser.add_argument('-e_weights','--enc_weights', type=str, default='', help='file path for encoder weights', required=False)
parser.add_argument('-e_freeze','--enc_freeze', type=lambda x: (str(x).lower() == 'true'), default=False, help='whether to freeze the encoder weights', required=False)
parser.add_argument('-lat_nonlin','--latent_nonlinearity', type=str, default='', help='type of to use prior to decoder', required=False)
parser.add_argument('-d_nh','--dec_n_hidden', type=int, nargs='+', default=[128], help='# of hidden units for decoder or list of hiddens for each layer', required=False)
parser.add_argument('-d_nl','--dec_n_layer', type=int, default=0, help='# of hidden layers for decoder', required=False)
parser.add_argument('-d_nonlin','--dec_nonlinearity', type=str, default='', help='type of nonlinearity for decoder', required=False)
parser.add_argument('-d_weights','--dec_weights', type=str, default='', help='file path for decoder weights', required=False)
parser.add_argument('-d_freeze','--dec_freeze', type=lambda x: (str(x).lower() == 'true'), default=False, help='whether to freeze the decoder weights', required=False)
parser.add_argument('-d_word_dist','--dec_word_dist', type=lambda x: (str(x).lower() == 'true'), default=False, help='whether to init decoder weights with training set word distributions', required=False)
parser.add_argument('-dis_nh','--dis_n_hidden', type=int, nargs='+', default=[128], help='# of hidden units for encoder or list of hiddens for each layer', required=False)
parser.add_argument('-dis_nl','--dis_n_layer', type=int, default=1, help='# of hidden layers for encoder, set to -1 if passing list of n_hiddens', required=False)
parser.add_argument('-dis_nonlin','--dis_nonlinearity', type=str, default='sigmoid', help='type of nonlinearity for discriminator', required=False)
parser.add_argument('-dis_y_weights','--dis_y_weights', type=str, default='', help='file path for discriminator_y weights', required=False)
parser.add_argument('-dis_z_weights','--dis_z_weights', type=str, default='', help='file path for discriminator_z weights', required=False)
parser.add_argument('-dis_freeze','--dis_freeze', type=lambda x: (str(x).lower() == 'true'), default=False, help='whether to freeze the encoder weights', required=False)
parser.add_argument('-include_w','--include_weights', type=str, nargs='*', default=[], help='weights to train on (default is all weights) -- all others are kept fixed; Ex: E.z_encoder D.decoder', required=False)
parser.add_argument('-eps','--epsilon', type=float, default=1e-8, help='epsilon param for Adam', required=False)
parser.add_argument('-mx_it','--max_iter', type=int, default=50001, help='max # of training iterations', required=False)
parser.add_argument('-train_stats_every','--train_stats_every', type=int, default=100, help='skip train_stats_every iterations between recording training stats', required=False)
parser.add_argument('-eval_stats_every','--eval_stats_every', type=int, default=100, help='skip eval_stats_every iterations between recording evaluation stats', required=False)
parser.add_argument('-ndim_y','--ndim_y', type=int, default=256, help='dimensionality of y - topic indicator', required=False)
parser.add_argument('-ndim_x','--ndim_x', type=int, default=2, help='dimensionality of p(x) - data distribution', required=False)
parser.add_argument('-saveto','--saveto', type=str, default='', help='path prefix for saving results', required=False)
parser.add_argument('-gpu','--gpu', type=int, default=-2, help='if/which gpu to use (-1: all, -2: None)', required=False)
parser.add_argument('-hybrid','--hybridize', type=lambda x: (str(x).lower() == 'true'), default=False, help='declaritive True (hybridize) or imperative False', required=False)
parser.add_argument('-full_npmi','--full_npmi', type=lambda x: (str(x).lower() == 'true'), default=False, help='whether to compute NPMI for full trajectory', required=False)
parser.add_argument('-eot','--eval_on_test', type=lambda x: (str(x).lower() == 'true'), default=False, help='whether to evaluate on the test set (True) or validation set (False)', required=False)
parser.add_argument('-verb','--verbose', type=lambda x: (str(x).lower() == 'true'), default=True, help='whether to print progress to stdout', required=False)
parser.add_argument('-dirich_alpha','--dirich_alpha', type=float, default=1e-1, help='param for Dirichlet prior', required=False)
parser.add_argument('-adverse','--adverse', type=lambda x: (str(x).lower() == 'true'), default=True, help='whether to turn on adverserial training (MMD or GAN). set to False if only train auto-encoder', required=False)
parser.add_argument('-update_enc','--update_enc', type=lambda x: (str(x).lower() == 'true'), default=True, help='whether to update encoder for unlabed_train_op()', required=False)
parser.add_argument('-labeled_loss_lambda','--labeled_loss_lambda', type=float, default=1.0, help='param for Dirichlet noise for label', required=False)
parser.add_argument('-train_mode','--train_mode', type=str, default='mmd', help="set to mmd or adv (for GAN)", required=False)
parser.add_argument('-kernel_alpha','--kernel_alpha', type=float, default=1.0, help='param for information diffusion kernel', required=False)
parser.add_argument('-recon_alpha','--recon_alpha', type=float, default=-1.0, help='multiplier of the reconstruction loss when combined with mmd loss', required=False)
parser.add_argument('-recon_alpha_adapt','--recon_alpha_adapt', type=float, default=-1.0, help='adaptively change recon_alpha so that [total loss = mmd + recon_alpha_adapt * recon loss], set to -1 if no adapt', required=False)
parser.add_argument('-dropout_p','--dropout_p', type=float, default=-1.0, help='dropout probability in encoder', required=False)
parser.add_argument('-l2_alpha','--l2_alpha', type=float, default=-1.0, help='alpha multipler for L2 regularization on latent vector', required=False)
parser.add_argument('-latent_noise','--latent_noise', type=float, default=0.0, help='proportion of dirichlet noise added to the latent vector after softmax', required=False)
parser.add_argument('-topic_decoder_weight','--topic_decoder_weight', type=lambda x: (str(x).lower() == 'true'), default=False, help='extract topic words based on decoder weights or decoder outputs', required=False)
parser.add_argument('-retrain_enc_only','--retrain_enc_only', type=lambda x: (str(x).lower() == 'true'), default=False, help='only retrain the encoder for reconstruction loss', required=False)
parser.add_argument('-l2_alpha_retrain','--l2_alpha_retrain', type=float, default=0.1, help='alpha multipler for L2 regularization on encoder output during retraining', required=False)
args = vars(parser.parse_args())

In [6]:
args

{'domain': 'wikitext-103',
 'data_path': '',
 'max_labels': 100,
 'max_labeled_samples': 10,
 'label_seed': None,
 'model': 'dirichlet',
 'description': '',
 'algorithm': 'standard',
 'batch_size': 256,
 'optim': 'Adam',
 'learning_rate': 0.0001,
 'weight_decay': 0.0,
 'enc_n_hidden': [128],
 'enc_n_layer': 1,
 'enc_nonlinearity': 'sigmoid',
 'enc_weights': '',
 'enc_freeze': False,
 'latent_nonlinearity': '',
 'dec_n_hidden': [128],
 'dec_n_layer': 0,
 'dec_nonlinearity': '',
 'dec_weights': '',
 'dec_freeze': False,
 'dec_word_dist': False,
 'dis_n_hidden': [128],
 'dis_n_layer': 1,
 'dis_nonlinearity': 'sigmoid',
 'dis_y_weights': '',
 'dis_z_weights': '',
 'dis_freeze': False,
 'include_weights': [],
 'epsilon': 1e-08,
 'max_iter': 50001,
 'train_stats_every': 100,
 'eval_stats_every': 100,
 'ndim_y': 256,
 'ndim_x': 2,
 'saveto': '',
 'gpu': -2,
 'hybridize': False,
 'full_npmi': False,
 'eval_on_test': False,
 'verbose': True,
 'dirich_alpha': 0.1,
 'adverse': True,
 'update_enc'

In [4]:
args['domain']

'twenty_news_sklearn'

In [10]:
sys.path.append('./examples/domains')

In [12]:
from wikitext103_wae import Wikitext103 as Domain

In [14]:
from models.dirichlet import Encoder, Decoder, Discriminator_y

In [16]:
from compute_op import Unsupervised as Compute

In [19]:
args['latent_noise'] >= 0.0 and args['latent_noise'] <= 1

True

In [20]:
args['description']

''

In [26]:
args['description'] = args['domain'] + '-' \
                    + args['algorithm'] + '-' \
                    + args['model'] + '-sup'

In [28]:
args['saveto'] = 'examples/results/' + args['description'].replace('-', '/')

In [29]:
saveto = args['saveto'] + '/' + \
    datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S/{}').format('')

In [30]:
os.path.exists(saveto)

False

In [31]:
os.makedirs(saveto)
os.makedirs(saveto + '/weights/encoder')
os.makedirs(saveto + '/weights/decoder')
os.makedirs(saveto + '/weights/discriminator_y')
os.makedirs(saveto + '/weights/discriminator_z')

In [33]:
os.path.realpath('compute_op.py')

'C:\\workspace\\Implementation\\w-lda\\compute_op.py'

In [34]:
os.path.join(saveto, 'compute_op.py')

'examples/results/wikitext/103/standard/dirichlet/sup/2019-12-19-10-12-59/compute_op.py'

In [35]:
shutil.copy(os.path.realpath('compute_op.py'), os.path.join(saveto, 'compute_op.py'))
shutil.copy(os.path.realpath('core.py'), os.path.join(saveto, 'core.py'))
shutil.copy(os.path.realpath('run.py'), os.path.join(saveto, 'run.py'))
shutil.copy(os.path.realpath('utils.py'), os.path.join(saveto, 'utils.py'))

'examples/results/wikitext/103/standard/dirichlet/sup/2019-12-19-10-12-59/utils.py'

In [37]:
model_file = args['model'] + '.py'
shutil.copy(os.path.realpath('models/' + model_file), os.path.join(saveto, model_file))

'examples/results/wikitext/103/standard/dirichlet/sup/2019-12-19-10-12-59/dirichlet.py'

In [41]:
args['saveto'] = saveto

In [44]:
with open(saveto + 'args.txt', 'w') as file:
    for key, val in args.items():
        if val != '':
            if isinstance(val, list) or isinstance(val, tuple):
                val = [str(v) for v in val]
                file.write('--' + str(key) + ' ' + ' '.join(val) + '\n')
            else:
                file.write('--' + str(key) + ' ' + str(val) + '\n')

In [49]:
args['gpu'] = 0

In [50]:
gpu_exists(args['gpu'])

True

In [52]:
args['description'] += ' (gpu' + str(args['gpu']) + ')'

In [53]:
args['description']

'wikitext-103-standard-dirichlet-sup (gpu0)'

In [54]:
pickle.dump(args, open(args['saveto'] + 'args.p', 'wb'))

In [55]:
Compute, Domain, Encoder, Decoder, Discriminator_y, args

(compute_op.Unsupervised,
 wikitext103_wae.Wikitext103,
 models.dirichlet.Encoder,
 models.dirichlet.Decoder,
 models.dirichlet.Discriminator_y,
 {'domain': 'wikitext-103',
  'data_path': '',
  'max_labels': 100,
  'max_labeled_samples': 10,
  'label_seed': None,
  'model': 'dirichlet',
  'description': 'wikitext-103-standard-dirichlet-sup (gpu0)',
  'algorithm': 'standard',
  'batch_size': 256,
  'optim': 'Adam',
  'learning_rate': 0.0001,
  'weight_decay': 0.0,
  'enc_n_hidden': [128],
  'enc_n_layer': 1,
  'enc_nonlinearity': 'sigmoid',
  'enc_weights': '',
  'enc_freeze': False,
  'latent_nonlinearity': '',
  'dec_n_hidden': [128],
  'dec_n_layer': 0,
  'dec_nonlinearity': '',
  'dec_weights': '',
  'dec_freeze': False,
  'dec_word_dist': False,
  'dis_n_hidden': [128],
  'dis_n_layer': 1,
  'dis_nonlinearity': 'sigmoid',
  'dis_y_weights': '',
  'dis_z_weights': '',
  'dis_freeze': False,
  'include_weights': [],
  'epsilon': 1e-08,
  'max_iter': 50001,
  'train_stats_every': 100,

In [56]:
print('\nSaving to: ' + args['saveto'])


Saving to: examples/results/wikitext/103/standard/dirichlet/sup/2019-12-19-10-12-59/


In [58]:
# model_ctx = gpu_helper(args['gpu'])
model_ctx = mx.gpu(args['gpu'])

In [62]:
Domain(batch_size=args['batch_size'],
       data_path=args['data_path'],
       ctx=model_ctx,
       saveto=args['saveto'])

FileNotFoundError: [Errno 2] No such file or directory: './data/wikitext-103/wikitext-103_tra.csr.npz'

https://blog.einstein.ai/the-wikitext-long-term-dependency-language-modeling-dataset/

In [65]:
# create directory for data
dataset = 'wikitext-103'
current_dir = os.getcwd()
current_dir

'C:\\workspace\\Implementation\\w-lda'

In [66]:
data_dir = os.path.join(current_dir, 'data')
data_dir

'C:\\workspace\\Implementation\\w-lda\\data'

In [68]:
print('Creating directory:', data_dir)
os.mkdir(data_dir)

Creating directory: C:\workspace\Implementation\w-lda\data


In [71]:
data_dir = os.path.join(current_dir, 'data', dataset)
# os.path.exists(data_dir)
os.mkdir(data_dir)

In [72]:
os.chdir(data_dir)
print('Current directory:', os.getcwd())

Current directory: C:\workspace\Implementation\w-lda\data\wikitext-103


In [73]:
# download data
os.system('curl -O '
  'https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-v1.zip')
os.system('unzip wikitext-103-v1.zip')

1

In [75]:
input_dir = os.path.join(data_dir, dataset)
input_dir

'C:\\workspace\\Implementation\\w-lda\\data\\wikitext-103\\wikitext-103'

In [76]:
train_file = 'wiki.train.tokens'
val_file = 'wiki.valid.tokens'
test_file = 'wiki.test.tokens'

In [77]:
# parse into documents
def is_document_start(line):
    if len(line) < 4:
        return False
    if line[0] is '=' and line[-1] is '=':
        if line[2] is not '=':
            return True
        else:
            return False
    else:
        return False

In [80]:
def token_list_per_doc(input_dir, token_file):
    lines_list = []
    line_prev = ''
    prev_line_start_doc = False
    with open(os.path.join(input_dir, token_file), 'r', encoding='utf-8') as f:
        for l in f:
            line = l.strip()
            if prev_line_start_doc and line:
                # the previous line should not have been start of a document!
                lines_list.pop()
                lines_list[-1] = lines_list[-1] + ' ' + line_prev

            if line:
                if is_document_start(line) and not line_prev:
                    lines_list.append(line)
                    prev_line_start_doc = True
                else:
                    lines_list[-1] = lines_list[-1] + ' ' + line
                    prev_line_start_doc = False
            else:
                prev_line_start_doc = False
            line_prev = line
    print("{} documents parsed!".format(len(lines_list)))
    return lines_list

In [81]:
train_doc_list = token_list_per_doc(input_dir, train_file)
val_doc_list = token_list_per_doc(input_dir, val_file)
test_doc_list = token_list_per_doc(input_dir, test_file)

28472 documents parsed!
60 documents parsed!
60 documents parsed!


In [87]:
from core import Data
from utils import reverse_dict
import scipy.sparse as sparse
import nltk

In [88]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jinma\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [89]:
from nltk.stem import WordNetLemmatizer
import re

In [90]:
token_pattern = re.compile(r'(?u)\b\w\w+\b')

class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in doc.split()
                if len(t) >= 2 and 
                   re.match('[a-z].*', t) and
                   re.match(token_pattern, t)]

import time
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [91]:
print('Lemmatizing and counting, this may take a few minutes...')
start_time = time.time()
vectorizer = CountVectorizer(input='content', analyzer='word',
                             stop_words='english', tokenizer=LemmaTokenizer(),
                             max_df=.8, min_df=3, max_features=20000)
train_vectors = vectorizer.fit_transform(train_doc_list)
val_vectors = vectorizer.transform(val_doc_list)
test_vectors = vectorizer.transform(test_doc_list)

# vocab_list = vectorizer.get_feature_names()
# vocab_size = len(vocab_list)
# print('vocab size:', vocab_size)
# print('Done. Time elapsed: {:.2f}s'.format(time.time() - start_time))

Lemmatizing and counting, this may take a few minutes...


C:\Users\jinma\AppData\Local\Continuum\anaconda3\envs\basic\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


NameError: name 'vocab_list' is not defined

In [92]:
vocab_list = vectorizer.get_feature_names()
vocab_size = len(vocab_list)
print('vocab size:', vocab_size)
print('Done. Time elapsed: {:.2f}s'.format(time.time() - start_time))

vocab size: 20000
Done. Time elapsed: 803.63s


In [94]:
def shuffle_and_dtype(vectors):
    idx = np.arange(vectors.shape[0])
    np.random.shuffle(idx)
    vectors = vectors[idx]
    vectors = sparse.csr_matrix(vectors, dtype=np.float32)
    print(type(vectors), vectors.dtype)
    return vectors

In [96]:
train_vectors = shuffle_and_dtype(train_vectors)
val_vectors = shuffle_and_dtype(val_vectors)
test_vectors = shuffle_and_dtype(test_vectors)

<class 'scipy.sparse.csr.csr_matrix'> float32
<class 'scipy.sparse.csr.csr_matrix'> float32
<class 'scipy.sparse.csr.csr_matrix'> float32


In [97]:
with open('vocab.txt', 'w', encoding='utf-8') as f:
    for item in vocab_list:
        f.write(item + '\n')

In [98]:
sparse.save_npz('wikitext-103_tra.csr.npz', train_vectors)
sparse.save_npz('wikitext-103_val.csr.npz', val_vectors)
sparse.save_npz('wikitext-103_test.csr.npz', test_vectors)

#### 다시

In [99]:
Domain(batch_size=args['batch_size'],
       data_path=args['data_path'],
       ctx=model_ctx,
       saveto=args['saveto'])

FileNotFoundError: [Errno 2] No such file or directory: './data/wikitext-103/wikitext-103_tra.csr.npz'

In [104]:
class Test(object):
    def __init__(self, a, b):
        self.a = a
        self.b = b
        self.load()
    def load(self):
        print('First')
        
class Tester(Test):
    def __init__(self, a, b):
        super(Tester, self).__init__(a, b)
    def load(self):
        print('Not First')

In [106]:
Tester(1, 2)

Not First


In [107]:
path = './data/wikitext-103'
features = 'BoW'
match_avitm = True

In [113]:
train_path = path + '/wikitext-103_tra.csr.npz'
test_path = path + '/wikitext-103_test.csr.npz'
vocab_path = path + '/vocab.txt'

In [114]:
train_csr = sparse.load_npz(train_path)

FileNotFoundError: [Errno 2] No such file or directory: './data/wikitext-103/wikitext-103_tra.csr.npz'

In [117]:
os.chdir(current_dir)
print('Current directory:', os.getcwd())

Current directory: C:\workspace\Implementation\w-lda


In [119]:
data = Domain(
    batch_size=args['batch_size'],
    data_path=args['data_path'],
    ctx=model_ctx,
    saveto=args['saveto'])

NOTE: Number of samples for test is smaller than batch_size (60<256). Duplicating samples to exceed batch_size.


In [120]:
print('train dimension = ', data.data['train'].shape)

train dimension =  (28472, 20000)


In [121]:
type(data.data['train'])

numpy.ndarray

In [122]:
mean_length = np.mean(np.sum(data.data['train'], axis=1))
mean_length

1392.214

In [123]:
vocab_size = data.data['train'].shape[1]

In [127]:
data.data['train_with_labels'] is None

True

In [128]:
args['recon_alpha']

-1.0

In [131]:
args['recon_alpha'] = 1.0 / (mean_length * np.log(vocab_size))

In [132]:
print('Setting recon_alpha to {}'.format(args['recon_alpha']))

Setting recon_alpha to 7.252802419767204e-05


### core, dirichlet

In [157]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss, v_measure_score

import mxnet as mx
from mxnet import gluon, io

import scipy.sparse as sparse
import json

In [ ]:
class ENet(gluon.HybridBlock):
    
    def __init__(self):
        super(ENet, self).__init__()
        
    def hybrid_forward(self, x):
        raise NotImplementedError(
            'Need to write your own Encoder that inherits from ENet. '
            'Put this file in models/.')
    
    def init_weights(self, weights=None):
        loaded = False
        source = 'keyword argument'
        if self.weights_file != '' and weights is None:
            try:
                self.load_params(self.weights_file, self.model_ctx)
                source = 'mxnet weights file: ' + self.weights_file
                print('NOTE: Loaded encoder weights from ' + source + '.')
                if self.freeze():
                    self.freeze_params()
                    print('NOTE: Froze encoder weights from ' + source + '.')
                weights = None
                loaded = None
            except:
                weights = pickle.load(open(self.weights_file, 'rb'))
                source = 'pickle file: ' + self.weights_file
        if weights is not None:
            assert self.n_layers == 0
            for p, w in zip(self.collect_params().values(), weights):
                if w is not None:

In [146]:
Enc = Encoder(
    model_ctx=model_ctx, 
    batch_size=args['batch_size'], 
    input_dim=args['ndim_x'], 
    ndim_y=args['ndim_y'],
    n_hidden=args['enc_n_hidden'], 
    n_layers=args['enc_n_layer'], 
    nonlin=args['enc_nonlinearity'], 
    weights_file=args['enc_weights'], 
    freeze=args['enc_freeze'], 
    latent_nonlin=args['latent_nonlinearity'])

NOTE: Encoder ignoring list of hiddens because n_layer >= 0. Just using first element.


In [156]:
Dec = Decoder(
    model_ctx=model_ctx, 
    batch_size=args['batch_size'], 
    output_dim=args['ndim_x'], 
    ndim_y=args['ndim_y'],
    n_hidden=[128], 
    n_layers=args['dec_n_layer'], 
    nonlin=args['dec_nonlinearity'],
    weights_file=args['dec_weights'], 
    freeze=args['dec_freeze'], 
    latent_nonlin=args['latent_nonlinearity'])



NOTE: Decoder ignoring list of hiddens because n_layer >= 0. Just using first element.


IndexError: list index out of range

In [150]:
args['dec_n_hidden'][0]

128

In [154]:
Dis_y = Discriminator_y(
    model_ctx=model_ctx, 
    batch_size=args['batch_size'], 
    ndim_y=args['ndim_y'],
    n_hidden=args['dis_n_hidden'], 
    n_layers=args['dis_n_layer'],
    nonlin=args['dis_nonlinearity'], 
    weights_file=args['dis_y_weights'],
    freeze=args['dis_freeze'], 
    latent_nonlin=args['latent_nonlinearity'])

NOTE: Decoder ignoring list of hiddens because n_layer >= 0. Just using first element.
NOTE: Latent z will be fed to decoder in logit-space (-inf,inf).
